# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagaimana yang sudah ditentukan.

### Dataset yang digunakan
Saya menggunakan dataset **Credit Scoring (synthetic)** yang disimpan di repository ini (format CSV).

- **Sumber data (tautan):**
  - GitHub (view): https://github.com/eko-andri-prasetyo/Eksperimen_SML_EkoAndriPrasetyo/blob/main/creditscoring_raw/creditscoring_raw.csv
  - GitHub (raw): https://raw.githubusercontent.com/eko-andri-prasetyo/Eksperimen_SML_EkoAndriPrasetyo/main/creditscoring_raw/creditscoring_raw.csv
- **Lokasi file raw di repo:** `creditscoring_raw/creditscoring_raw.csv`
- **Jumlah data:** 2000 baris, 11 kolom (10 fitur + 1 target)
- **Target:** `target` (0 = tidak default, 1 = default)
- **Fitur:**
  - Numerik: `age`, `monthly_income`, `loan_amount`, `tenure_months`, `num_credit_lines`, `has_previous_default`
  - Kategorikal: `job_type`, `education_level`, `city`, `marital_status`
- **Catatan kualitas data:** terdapat missing value pada `num_credit_lines`, `job_type`, dan `education_level` sehingga perlu imputasi pada tahap preprocessing.

Catatan: Dataset ini bersifat **synthetic (dibuat untuk kebutuhan latihan/proyek)** dan disediakan di repo agar dapat diakses reviewer dengan mudah.


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [ ]:
RAW_PATH = Path("creditscoring_raw/creditscoring_raw.csv")
df = pd.read_csv(RAW_PATH)

print("Shape:", df.shape)
df.head()


# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
# Info ringkas dataset
df.info()

# Statistik deskriptif kolom numerik
df.describe().T

# Cek missing values
missing = df.isna().sum().sort_values(ascending=False)
missing[missing > 0]

# Distribusi target (jika ada)
if "target" in df.columns:
    df["target"].value_counts(dropna=False)


# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam pelatihan model.

Output tahap ini adalah dataset yang **sudah siap dipakai untuk training**, sehingga tahap modelling (Kriteria 2) **tidak perlu melakukan preprocessing lagi**.


In [ ]:
# ---- CONFIG ----
TARGET_COL = "target"
OUT_DIR = Path("preprocessing/creditscoring_preprocessing")
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_PATH = OUT_DIR / "creditscoring_preprocessed.csv"

if TARGET_COL not in df.columns:
    raise ValueError(
        f"Kolom target '{TARGET_COL}' tidak ditemukan. Kolom tersedia: {list(df.columns)}"
    )

# ---- BASIC CLEANING ----
df_prep = df.drop_duplicates().copy()

# imputasi sederhana: numerik -> median, kategorikal -> modus
num_cols = [c for c in df_prep.columns if c != TARGET_COL and pd.api.types.is_numeric_dtype(df_prep[c])]
cat_cols = [c for c in df_prep.columns if c != TARGET_COL and not pd.api.types.is_numeric_dtype(df_prep[c])]

for col in num_cols:
    df_prep[col] = df_prep[col].fillna(df_prep[col].median())

for col in cat_cols:
    mode_val = df_prep[col].mode(dropna=True)
    df_prep[col] = df_prep[col].fillna(mode_val.iloc[0] if len(mode_val) else "")

# pastikan target integer (0/1)
df_prep[TARGET_COL] = df_prep[TARGET_COL].astype(int)

# ---- ENCODING (One-Hot) ----
X = df_prep.drop(columns=[TARGET_COL])
y = df_prep[TARGET_COL]

X_enc = pd.get_dummies(X, drop_first=False)

df_out = pd.concat([X_enc, y], axis=1)

# ---- SAVE ----
df_out.to_csv(OUT_PATH, index=False)

print("Saved:", OUT_PATH)
print("Output shape:", df_out.shape)
df_out.head()
